<a href="https://colab.research.google.com/github/oenereaty/MP2024/blob/main/MP2024_wk9_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [3]:
!pip uninstall mp2024pkg -y
!pip install git+https://github.com/guebin/mp2024pkg.git

  Cloning https://github.com/guebin/mp2024pkg.git to /tmp/pip-req-build-sy1dscjm
  Running command git clone --filter=blob:none --quiet https://github.com/guebin/mp2024pkg.git /tmp/pip-req-build-sy1dscjm
  Resolved https://github.com/guebin/mp2024pkg.git to commit 2adb5fc04e589895edd76ec5da87aa37bf0dd70b
  Preparing metadata (setup.py) ... done
  Created wheel for mp2024pkg: filename=mp2024pkg-1.0-py3-none-any.whl size=4588 sha256=b551f1cec0a3c15857932c7b9468ce637dd58bbb9e329df72eba48d2026d220e
  Stored in directory: /tmp/pip-ephem-wheel-cache-vqe101wf/wheels/34/e6/bd/d7e7b36b46f974e90ffd4f6308065d303cde840f9c0e0b0892
Successfully built mp2024pkg


In [4]:
pip install datasets pytorchvideo transformers huggingface_hub torch torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.1 MB/s eta 0:00:00
  Created wheel for pytorchvideo: filename=pytorchvideo-0.1.5-py3-none-any.whl size=188685 sha256=7ce228f088069921d33bdcc97e8d67e2364dbbc99a0b54d3c19b8d

In [5]:
import transformers
import datasets
import huggingface_hub
import torch
import torchvision
import pytorchvideo.data
import PIL
import tarfile
import mp2024pkg as mp

# Model 입력 파악

## 예제1

In [5]:
model1 = transformers.AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model의 `config`를 통해 기본 정보를 파악하자

In [6]:
model1.config

DistilBertConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "distilbert/distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.46.2",
  "vocab_size": 30522
}

model의 입력을 파악하기 위해 `forward`를 보자

In [7]:
model1.forward?

In [8]:
model1(
    input_ids = torch.tensor([[1,2,3,4], [2,3,4,5]]),
    labels = torch.tensor([0,0])
)

SequenceClassifierOutput(loss=tensor(0.7484, grad_fn=<NllLossBackward0>), logits=tensor([[0.0141, 0.1216],
        [0.0163, 0.1238]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [9]:
torch.tensor(1.0).long().float()

tensor(1.)

사용예시1 – 입력나열, loss O

In [10]:
model1(
    input_ids = torch.tensor([[1,2,3,4], [2,3,4,5]]),
    labels = torch.tensor([0,0])
)

# model은 입력값을 list로 받길 기대하고 있다

SequenceClassifierOutput(loss=tensor(0.7484, grad_fn=<NllLossBackward0>), logits=tensor([[0.0141, 0.1216],
        [0.0163, 0.1238]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

사용예시2 – **딕셔너리, loss O


`model(**dict)`

In [11]:
model1_input = dict(
    input_ids = torch.tensor([[1,2,3,4], [2,3,4,5]]),
    labels = torch.tensor([0,0])
)
model1(**model1_input)

SequenceClassifierOutput(loss=tensor(0.7484, grad_fn=<NllLossBackward0>), logits=tensor([[0.0141, 0.1216],
        [0.0163, 0.1238]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

사용예시3 – 입력나열, loss X

`label` 미전달

In [12]:
model1(
    input_ids = torch.tensor([[1,2,3,4], [2,3,4,5]])
)

SequenceClassifierOutput(loss=None, logits=tensor([[0.0141, 0.1216],
        [0.0163, 0.1238]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

사용예시4 – `**딕셔너리`, loss X

In [13]:
model1_input = dict(
    input_ids = torch.tensor([[1,2,3,4], [2,3,4,5]])
)
model1(**model1_input)

SequenceClassifierOutput(loss=None, logits=tensor([[0.0141, 0.1216],
        [0.0163, 0.1238]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

사용예시5 – 초간단, loss X


`input_ids` 또한 쓰지 않음

In [14]:
model1(torch.tensor([[1,2,3,4], [2,3,4,5]]))

SequenceClassifierOutput(loss=None, logits=tensor([[0.0141, 0.1216],
        [0.0163, 0.1238]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

## 예제2: 이미지 분류

In [15]:
model2 = transformers.AutoModelForImageClassification.from_pretrained(
    "google/vit-base-patch16-224-in21k",
    num_labels=3 # 그냥 대충 3이라고 했음.. 별 이유는 없음
)

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
model2.config

ViTConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "google/vit-base-patch16-224-in21k",
  "architectures": [
    "ViTModel"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "image_size": 224,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": true,
  "transformers_version": "4.46.2"
}

In [17]:
# mp.tab(model2)
# -- 결과가 너무 길어서 생략

모델의 입력 파악

In [18]:
# model2.forward?
# -- 결과가 너무 길어서 생략

# pixel_values와 labels 를 함께 주면 loss 계산
# pixel_values 만 주면 logits 값만 계산

In [19]:
torch.randn(10,10)

tensor([[ 0.5915,  0.6178,  0.7395,  2.0261, -0.3492, -0.1939,  0.0035, -0.4283,
         -1.5767,  0.1022],
        [-1.2435, -1.6458, -0.1702, -0.9070, -0.8986, -1.9594,  0.8944, -0.9543,
         -0.1981,  0.7166],
        [-2.3710, -1.4357, -0.1194, -1.0185,  0.5096, -0.6727, -1.5984, -1.2850,
          3.0481,  0.1258],
        [-0.0979, -1.1624,  0.1469, -0.1370,  0.0195,  0.7742,  0.6824,  1.2244,
          0.8756,  0.5296],
        [-0.1885, -0.9354,  0.3221,  1.2177,  2.3586,  1.1952, -0.8615,  1.0459,
          0.6977, -0.7984],
        [ 0.0591, -0.8777, -1.2726,  0.2785, -1.6088,  0.4488, -0.3060, -1.0195,
         -0.6041,  0.2126],
        [ 1.0106, -0.1946, -1.0989, -1.7441,  0.3103,  0.6899,  0.2200, -1.8578,
         -0.0105, -1.0647],
        [ 0.9198,  0.5347, -0.9412,  1.8419,  0.0790,  0.4687,  0.8544,  0.6011,
          1.0595,  0.0599],
        [-0.1579,  0.3855, -0.6835, -0.0170,  0.6202,  0.6278, -0.3972, -1.4888,
         -2.6272, -0.5978],
        [ 0.5679,  

In [20]:
torch.randn(2,3,64,64)

# 배치 2개, 채널 3개, 높이 64, 너비 64

tensor([[[[-1.1141,  1.0339,  0.1617,  ..., -1.9182,  0.1505,  0.8766],
          [-1.3824,  0.4767,  1.2433,  ...,  0.0809,  0.6620, -1.2000],
          [ 0.1735,  0.1837, -0.8848,  ..., -1.0202,  2.1299, -0.6053],
          ...,
          [ 0.2415,  0.6624, -0.9550,  ..., -0.8109, -1.3252, -0.8688],
          [-0.4385,  0.7993,  0.6692,  ...,  0.8195,  0.2956, -0.8402],
          [-0.2249,  1.4491, -0.7939,  ..., -0.9094, -0.2123, -0.5587]],

         [[ 1.1844, -0.2728, -0.1443,  ...,  2.3645, -0.3643, -0.5421],
          [-0.1315,  1.5448,  0.6138,  ...,  1.5511, -1.4895, -1.4049],
          [ 0.2837, -1.3496, -0.6900,  ..., -0.7621, -0.0091,  0.2636],
          ...,
          [ 1.3375,  0.2884, -1.0860,  ...,  0.4986, -0.0429,  0.0264],
          [-0.7514, -0.3469,  0.1481,  ...,  0.1655,  0.3741, -1.3011],
          [-0.5350, -0.6811,  0.0459,  ...,  0.4035, -0.1917,  0.4343]],

         [[-0.0174, -0.3915,  1.5148,  ...,  0.9664,  0.0640,  0.1783],
          [ 1.4913, -0.2443,  

사용예시1 – 입력나열, loss O



In [21]:
torch.random.manual_seed(42)
model2(
    pixel_values = torch.randn(2,3,224,224),
    labels = torch.tensor([0,1])
)

# 이미지 크기 올바르게 넣어줘!

ImageClassifierOutput(loss=tensor(1.1343, grad_fn=<NllLossBackward0>), logits=tensor([[-0.1007,  0.0399,  0.0358],
        [-0.0711,  0.0074,  0.0145]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [22]:
torch.random.manual_seed(42)
model2(
    pixel_values = torch.randn(2,3,24,224),
    labels = torch.tensor([0,1])
)

# 이미지 크기 올바르게 넣어줘!

ValueError: Input image size (24*224) doesn't match model (224*224).

사용예시2 – `**딕셔너리`, loss O

In [23]:
torch.random.manual_seed(42)
model2_input = dict(
    pixel_values = torch.randn(2,3,224,224),
    labels = torch.tensor([0,1])
)
model2(**model2_input)

ImageClassifierOutput(loss=tensor(1.1343, grad_fn=<NllLossBackward0>), logits=tensor([[-0.1007,  0.0399,  0.0358],
        [-0.0711,  0.0074,  0.0145]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

사용예시3 – 입력나열, loss X

In [24]:
torch.random.manual_seed(42)
model2(
    pixel_values = torch.randn(2,3,224,224)
)

ImageClassifierOutput(loss=None, logits=tensor([[-0.1007,  0.0399,  0.0358],
        [-0.0711,  0.0074,  0.0145]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

사용예시4 – `**딕셔너리`, loss X



In [25]:
torch.random.manual_seed(42)
model2_input = dict(
    pixel_values = torch.randn(2,3,224,224),
)
model2(**model2_input)

ImageClassifierOutput(loss=None, logits=tensor([[-0.1007,  0.0399,  0.0358],
        [-0.0711,  0.0074,  0.0145]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

사용예시5 – 초간단, loss X



In [26]:
torch.random.manual_seed(42)
model2(torch.randn(2,3,224,224))

ImageClassifierOutput(loss=None, logits=tensor([[-0.1007,  0.0399,  0.0358],
        [-0.0711,  0.0074,  0.0145]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

## 예제3: 동영상

In [27]:
model3 = transformers.VideoMAEForVideoClassification.from_pretrained(
    "MCG-NJU/videomae-base",
)

config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/377M [00:00<?, ?B/s]

Some weights of VideoMAEForVideoClassification were not initialized from the model checkpoint at MCG-NJU/videomae-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
model3.config

VideoMAEConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "MCG-NJU/videomae-base",
  "architectures": [
    "VideoMAEForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.0,
  "decoder_hidden_size": 384,
  "decoder_intermediate_size": 1536,
  "decoder_num_attention_heads": 6,
  "decoder_num_hidden_layers": 4,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 224,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "videomae",
  "norm_pix_loss": true,
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_frames": 16,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": true,
  "torch_dtype": "float32",
  "transformers_version": "4.46.2",
  "tubelet_size": 2,
  "use_mean_pooling": false
}

image_size: 224 고정, num_frames: 16 고정

모델의 입력 파악

In [29]:
# model3.forward?
# -- 결과가 너무 길어서 생략
# pixel_values 는 float(실수)형으로 입력해라
# 배치사이즈, 프레임, 채널, h, w 순서로 입력해라
# 라벨(label)은 배치사이즈만큼 int형으로 입력해라

사용예시1 – 입력나열, loss O



In [30]:
torch.random.manual_seed(42)
model3(
    pixel_values = torch.randn(4,16,3,224,224),
    labels = torch.tensor([0,1,0,1])
)

ImageClassifierOutput(loss=tensor(0.7254, grad_fn=<NllLossBackward0>), logits=tensor([[-0.7084, -0.2919],
        [-0.5846, -0.1854],
        [-0.6635, -0.2402],
        [-0.7193, -0.3811]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

사용예시2 – `**딕셔너리`, loss O



In [31]:
torch.random.manual_seed(42)
model3_input = dict(
    pixel_values = torch.randn(4,16,3,224,224),
    labels = torch.tensor([0,1,0,1])
)
model3(**model3_input)

ImageClassifierOutput(loss=tensor(0.7254, grad_fn=<NllLossBackward0>), logits=tensor([[-0.7084, -0.2919],
        [-0.5846, -0.1854],
        [-0.6635, -0.2402],
        [-0.7193, -0.3811]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

사용예시3 – 입력나열, loss X



In [ ]:
torch.random.manual_seed(42)
model3(
    pixel_values = torch.randn(4,16,3,224,224)
)

사용예시4 – `**딕셔너리`, loss X



In [ ]:
torch.random.manual_seed(42)
model3_input = dict(
    pixel_values = torch.randn(4,16,3,224,224),
)
model3(**model3_input)

사용예시5 – 초간단, loss X



In [ ]:
torch.random.manual_seed(42)
model3(torch.randn(4,16,3,224,224))

# 모델 사용 연습

## 텍스트

In [1]:
model1 = transformers.AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2
)
tokenizer = transformers.AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

NameError: name 'transformers' is not defined

### 예제: `imdb`

In [6]:
imdb = datasets.load_dataset('imdb')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [7]:
model1.forward(torch.tensor(tokenizer(d['text'])['input_ids']))

# padding을 하지 않아 에러 발생

NameError: name 'model1' is not defined

In [ ]:
# 풀이 1
model1(torch.tensor(tokenizer(d['text'], padding=True)['input_ids']))

In [ ]:
# 풀이 2
model1(tokenizer(d['text'], padding=True, return_tensors="pt")['input_ids'])

In [ ]:
torch.tensor(tokenizer(d['text'], padding=True)['input_ids'])

### 예제2: `emotion`

In [ ]:
emotion = datasets.load_dataset('emotion')

In [ ]:
d = emotion['train'].select(range(3))
d

In [ ]:
model1(torch.tensor(tokenizer(d['text'],padding=True)['input_ids']))

### 예제3: mbti

In [ ]:
# mbti1.csv 파일 다운로드
!wget https://raw.githubusercontent.com/guebin/MP2024/refs/heads/main/posts/mbti_1.csv

In [ ]:
d = datasets.Dataset.from_csv("mbti_1.csv").select(range(3))
d

실패한 코드

In [ ]:
model1(torch.tensor(tokenizer(d['posts'],padding=True)['input_ids']))

`padding`까지 했는데 왜지?

In [ ]:
mp.show_list(
    tokenizer(d['posts'],padding=True)['input_ids']
)

In [ ]:
mp.show_list(
    tokenizer(d['posts'],truncation=True)['input_ids']
)

In [ ]:
torch.tensor(tokenizer(d['posts'], padding=True)['input_ids']).shape

torch.Size([3, 2102])

흠 사이즈가 너무 크다 -> 좀 깍둑썰기 해야 할 듯

=> `padding`이 아니라 `truncation` 필요!

In [ ]:
# 풀이 1

model1(torch.tensor(tokenizer(d['posts'],truncation=True)['input_ids']))
#model1(tokenizer(d['posts'],truncation=True,return_tensors="pt")['input_ids'])

(풀이2) –모델설정변경 (퀴즈5, 모델의 프레임수를 4로 바꾸는 예제에서 사용한 테크닉)

In [ ]:
config = transformers.AutoConfig.from_pretrained(
    "distilbert/distilbert-base-uncased"
)
config

설정값 변경

In [ ]:
config.max_position_embeddings = 2200

설정값으로 모델 불러오기

In [ ]:
model1_large = transformers.AutoModelForSequenceClassification.from_config(
    config=config
)

# 그동안의 코드와 다른 점은 이 코드는 `from_config`를 사용함

모델 사용

In [ ]:
# 방법 1

model1_large(torch.tensor(tokenizer(d['posts'],padding=True)['input_ids']))

In [ ]:
# 방법 2

model1_large(**tokenizer(d['posts'],padding=True,return_tensors="pt"))

### 예제 3: sms_spam

In [ ]:
sms_spam = datasets.load_dataset('sms_spam')['train'].train_test_split(test_size=0.2, seed=42)
sms_spam

In [ ]:
d = sms_spam['train'].select(range(3))
d

풀이

In [ ]:
tokenizer(d['sms'], padding=True, return_tensors="pt")['input_ids']

In [ ]:
model1(**tokenizer(d['sms'],padding=True,return_tensors="pt"))

## 이미지

In [16]:
model2 = transformers.AutoModelForImageClassification.from_pretrained(
    "google/vit-base-patch16-224-in21k",
    num_labels=3 # 그냥 대충 3이라고 했음.. 별 이유는 없음
)

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model2.config

### 예제1: food101

In [8]:
d = datasets.load_dataset("food101", split="train[:4]")
d

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

train-00000-of-00008.parquet:   0%|          | 0.00/490M [00:00<?, ?B/s]

train-00001-of-00008.parquet:   0%|          | 0.00/464M [00:00<?, ?B/s]

train-00002-of-00008.parquet:   0%|          | 0.00/472M [00:00<?, ?B/s]

train-00003-of-00008.parquet:   0%|          | 0.00/464M [00:00<?, ?B/s]

...

validation-00001-of-00003.parquet:   0%|          | 0.00/413M [00:00<?, ?B/s]

validation-00002-of-00003.parquet:   0%|          | 0.00/426M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/75750 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/25250 [00:00<?, ? examples/s]

Dataset({
    features: ['image', 'label'],
    num_rows: 4
})

예비학습: `torchvision.transforms` 에서 제공하는 기능들은 배치처리가 가능한가?

In [9]:
to_tensor = torchvision.transforms.ToTensor()

In [10]:
to_tensor(d['image'][0])

tensor([[[0.1216, 0.1137, 0.1098,  ..., 0.0039, 0.0039, 0.0000],
         [0.1255, 0.1216, 0.1176,  ..., 0.0039, 0.0039, 0.0000],
         [0.1294, 0.1255, 0.1255,  ..., 0.0039, 0.0000, 0.0000],
         ...,
         [0.2588, 0.2745, 0.2863,  ..., 0.3765, 0.3804, 0.3922],
         [0.2353, 0.2471, 0.2667,  ..., 0.3373, 0.3373, 0.3373],
         [0.2235, 0.2275, 0.2471,  ..., 0.3333, 0.3176, 0.3059]],

        [[0.1373, 0.1294, 0.1255,  ..., 0.1020, 0.1020, 0.0980],
         [0.1412, 0.1373, 0.1333,  ..., 0.1020, 0.1020, 0.0980],
         [0.1451, 0.1412, 0.1412,  ..., 0.1020, 0.0980, 0.0980],
         ...,
         [0.2471, 0.2627, 0.2745,  ..., 0.3647, 0.3765, 0.3882],
         [0.2235, 0.2353, 0.2549,  ..., 0.3255, 0.3333, 0.3333],
         [0.2118, 0.2157, 0.2353,  ..., 0.3216, 0.3137, 0.3020]],

        [[0.1412, 0.1333, 0.1373,  ..., 0.0902, 0.0902, 0.0863],
         [0.1451, 0.1412, 0.1451,  ..., 0.0902, 0.0902, 0.0863],
         [0.1569, 0.1529, 0.1529,  ..., 0.0902, 0.0863, 0.

In [11]:
to_tensor(d['image'])

TypeError: pic should be PIL Image or ndarray. Got <class 'list'>

In [20]:
compose = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Resize((224,224))
])

In [13]:
torch.stack(list(map(compose,d['image'])),axis=0).shape

torch.Size([4, 3, 224, 224])

```python
torchvision.transforms.ToTensor()
```

얘도 함수의 역할을 하기 때문에 입력값을 [0],.. 로 받을 수밖에 없음

In [14]:
torch.stack(list(map(compose,d['image'])),axis=0).shape

torch.Size([4, 3, 224, 224])

In [17]:
model2.forward(
    torch.stack(list(map(compose,d['image'])),axis=0)
)

ImageClassifierOutput(loss=None, logits=tensor([[-0.1559, -0.1789,  0.0660],
        [-0.1492, -0.0505,  0.0067],
        [-0.1232, -0.0359, -0.0004],
        [ 0.1671,  0.1116, -0.1124]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [18]:
compose(d['image'][0]).shape

torch.Size([3, 224, 224])

In [23]:
compose(d['image'][0]).unsqueeze(0).shape

torch.Size([1, 3, 224, 224])

In [21]:
compose(d['image'][0]).reshape(1,3,224,224).shape

torch.Size([1, 3, 224, 224])

In [27]:
torch.stack(list(map(compose, d['image'])),axis=0)

# ['image'] 의 전체 원소를 tensor화 하기

tensor([[[[1.2233e-01, 1.1662e-01, 1.1591e-01,  ..., 5.9697e-03,
           4.1184e-03, 1.4459e-03],
          [1.3144e-01, 1.2900e-01, 1.2662e-01,  ..., 3.7239e-03,
           1.6477e-03, 1.0114e-03],
          [1.4253e-01, 1.3797e-01, 1.3386e-01,  ..., 3.6490e-03,
           1.7565e-03, 8.2576e-04],
          ...,
          [2.2425e-01, 2.1619e-01, 2.4810e-01,  ..., 3.5705e-01,
           3.7023e-01, 3.5503e-01],
          [2.6097e-01, 2.5844e-01, 2.3292e-01,  ..., 3.4785e-01,
           3.6223e-01, 3.5982e-01],
          [2.4081e-01, 2.6132e-01, 2.5348e-01,  ..., 3.3631e-01,
           3.4241e-01, 3.3812e-01]],

         [[1.3802e-01, 1.3231e-01, 1.3160e-01,  ..., 1.0401e-01,
           1.0216e-01, 9.9485e-02],
          [1.4713e-01, 1.4469e-01, 1.4231e-01,  ..., 1.0176e-01,
           9.8946e-02, 9.5119e-02],
          [1.5821e-01, 1.5365e-01, 1.4954e-01,  ..., 1.0026e-01,
           9.2966e-02, 8.6574e-02],
          ...,
          [2.1549e-01, 2.0744e-01, 2.4257e-01,  ..., 3.3113

### 예제2: `beans`

In [28]:
beans = datasets.load_dataset('beans')
d = beans['train'].select(range(4))
d

README.md:   0%|          | 0.00/4.95k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/144M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/18.5M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1034 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/133 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/128 [00:00<?, ? examples/s]

Dataset({
    features: ['image_file_path', 'image', 'labels'],
    num_rows: 4
})

In [39]:
torch.stack(list(map(compose, d['image'])),axis=0)

tensor([[[[2.1330e-01, 2.1489e-01, 2.1957e-01,  ..., 4.8770e-01,
           5.9313e-01, 3.7533e-01],
          [1.9653e-01, 2.0084e-01, 2.0985e-01,  ..., 4.8325e-01,
           4.8821e-01, 3.6766e-01],
          [1.8401e-01, 1.8347e-01, 1.9641e-01,  ..., 3.8328e-01,
           3.1906e-01, 3.8659e-01],
          ...,
          [2.3024e-01, 2.2410e-01, 2.0937e-01,  ..., 4.5888e-01,
           4.7549e-01, 4.5733e-01],
          [2.2128e-01, 2.1116e-01, 2.0613e-01,  ..., 6.5682e-01,
           5.9267e-01, 5.6779e-01],
          [2.1027e-01, 2.0477e-01, 1.9769e-01,  ..., 6.3248e-01,
           5.8060e-01, 5.9946e-01]],

         [[1.4426e-01, 1.4252e-01, 1.2706e-01,  ..., 3.1517e-01,
           4.2072e-01, 1.9735e-01],
          [1.3409e-01, 1.3341e-01, 1.2051e-01,  ..., 3.1911e-01,
           3.3887e-01, 2.1554e-01],
          [1.2632e-01, 1.2237e-01, 1.1324e-01,  ..., 2.3168e-01,
           1.7156e-01, 2.2994e-01],
          ...,
          [1.1510e-01, 1.0843e-01, 9.2029e-02,  ..., 3.8736

In [29]:
model2(torch.stack(list(map(compose,d['image'])),axis=0))

ImageClassifierOutput(loss=None, logits=tensor([[ 0.0190, -0.0739, -0.0278],
        [ 0.0442, -0.0582,  0.0302],
        [-0.0127,  0.0015,  0.0003],
        [-0.0637, -0.0178, -0.0641]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

## 동영상

In [40]:
model3 = transformers.VideoMAEForVideoClassification.from_pretrained(
    "MCG-NJU/videomae-base",
)

config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/377M [00:00<?, ?B/s]

Some weights of VideoMAEForVideoClassification were not initialized from the model checkpoint at MCG-NJU/videomae-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 예제1: UCF101_subset

In [41]:
file_path = huggingface_hub.hf_hub_download(
    repo_id="sayakpaul/ucf101-subset",
    filename="UCF101_subset.tar.gz",
    repo_type="dataset"
)
# file_path는 다운로드한 압축파일이 존재하는 경로와 파일명이 string으로 저장되어있음.
with tarfile.open(file_path) as t:
     t.extractall("./data") # 여기에서 "."은 현재폴더라는 의미

UCF101_subset.tar.gz:   0%|          | 0.00/171M [00:00<?, ?B/s]

In [42]:
mp.tree("./data")

└── UCF101_subset
    ├── test
    │   ├── ApplyEyeMakeup
    │   │   ├── UCF101
    │   │   ├── v_ApplyEyeMakeup_g03_c01.avi
    │   │   └── ...
    │   │   └── v_ApplyEyeMakeup_g23_c06.avi
    │   ├── ApplyLipstick
    │   │   ├── UCF101
    │   │   ├── v_ApplyLipstick_g14_c01.avi
    │   │   └── ...
    │   │   └── v_ApplyLipstick_g16_c04.avi
    │   └── ...
    │   └── BenchPress
    │       ├── UCF101
    │       ├── v_BenchPress_g05_c02.avi
    │       └── ...
    │       └── v_BenchPress_g25_c06.avi
    ├── train
    │   ├── ApplyEyeMakeup
    │   │   ├── UCF101
    │   │   ├── v_ApplyEyeMakeup_g02_c03.avi
    │   │   └── ...
    │   │   └── v_ApplyEyeMakeup_g25_c07.avi
    │   ├── ApplyLipstick
    │   │   ├── UCF101
    │   │   ├── v_ApplyLipstick_g01_c02.avi
    │   │   └── ...
    │   │   └── v_ApplyLipstick_g24_c05.avi
    │   └── ...
    │   └── BenchPress
    │       ├── UCF101
    │       ├── v_BenchPress_g01_c05.avi
    │       └── ...
    │       └── v_BenchPress_g24_c

In [46]:
video_path = "./data/UCF101_subset/test/BenchPress/v_BenchPress_g05_c02.avi"
video = pytorchvideo.data.encoded_video.EncodedVideo.from_path(video_path).get_clip(0, float('inf'))['video']
video.shape

# 3차원, 프레임 갯수 67, 240x320 사이즈

torch.Size([3, 67, 240, 320])

In [44]:
video

tensor([[[[161., 161., 161.,  ..., 157., 157., 157.],
          [160., 160., 160.,  ..., 156., 156., 156.],
          [156., 156., 156.,  ..., 155., 155., 155.],
          ...,
          [157., 157., 157.,  ..., 116., 116., 116.],
          [157., 157., 157.,  ..., 116., 116., 116.],
          [157., 157., 157.,  ..., 116., 116., 116.]],

         [[160., 160., 160.,  ..., 157., 157., 157.],
          [156., 156., 156.,  ..., 156., 156., 156.],
          [151., 151., 151.,  ..., 155., 155., 155.],
          ...,
          [157., 157., 157.,  ..., 116., 116., 116.],
          [157., 157., 157.,  ..., 116., 116., 116.],
          [157., 157., 157.,  ..., 116., 116., 116.]],

         [[160., 160., 160.,  ..., 157., 157., 157.],
          [156., 156., 156.,  ..., 156., 156., 156.],
          [151., 151., 151.,  ..., 155., 155., 155.],
          ...,
          [157., 157., 157.,  ..., 116., 116., 116.],
          [157., 157., 157.,  ..., 116., 116., 116.],
          [157., 157., 157.,  ...

In [45]:
model3.forward?

`torch.Size` 를 `reshape`하고 싶어!

In [48]:
# 1st: torch.Size의 요소 순서 교체
video.permute(1,0,2,3).shape

torch.Size([67, 3, 240, 320])

In [53]:
# 2nd 방법1: 요소를 원하는 구간까지 잘라(?)
video.permute(1,0,2,3)[:16,:,:224,:224].unsqueeze(0).shape

torch.Size([1, 16, 3, 224, 224])

In [56]:
# 2nd 방법2: 요소를 원하는 구간까지 잘라(?)
video.permute(1,0,2,3)[:16,:,:224,:224].reshape(1,16,3,224,224).shape

torch.Size([1, 16, 3, 224, 224])

In [57]:
# 3rd: model에 넣는다
model3(
    #video.permute(1,0,2,3)[:16,:,:224,:224].unsqueeze(0)
    #video.permute(1,0,2,3)[:16,:,:224,:224].reshape(1,16,3,224,224)
    torch.stack([video.permute(1,0,2,3)[:16,:,:224,:224]])
)

ImageClassifierOutput(loss=None, logits=tensor([[ 0.4014, -0.1926]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [58]:
# 3rd: model에 넣는다
model3(
    video.permute(1,0,2,3)[:16,:,:224,:224].unsqueeze(0)
    #video.permute(1,0,2,3)[:16,:,:224,:224].reshape(1,16,3,224,224)
    # torch.stack([video.permute(1,0,2,3)[:16,:,:224,:224]])
)

ImageClassifierOutput(loss=None, logits=tensor([[ 0.4014, -0.1926]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [59]:
# 3rd: model에 넣는다
model3(
    # video.permute(1,0,2,3)[:16,:,:224,:224].unsqueeze(0)
    video.permute(1,0,2,3)[:16,:,:224,:224].reshape(1,16,3,224,224)
    # torch.stack([video.permute(1,0,2,3)[:16,:,:224,:224]])
)

ImageClassifierOutput(loss=None, logits=tensor([[ 0.4014, -0.1926]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)